In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#to return a number with fixed numebr of significant digits

from math import log10 , floor

def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

In [ ]:
def open_train_cup(path):
    
    df = pd.read_csv(path, skiprows=7, header = None)                       
    df = df.drop([0], axis = 1)
    df = df.rename(columns={
        1: 'var_1',
        2: 'var_2',
        3: 'var_3',
        4: 'var_4',
        5: 'var_5',
        6: 'var_6',
        7: 'var_7',
        8: 'var_8',
        9: 'var_9',
        10: 'target_1',
        11: 'target_2' 
    })

    X = df.iloc[:, :9].values
    
    Y = df.iloc[:, 9:11].values
    
#     y_1 = df.iloc[:, 9:10].values.flatten()
    
#     y_2 = df.iloc[:, 10:11].values.flatten()
    
    return df, X, Y 


def open_blind_test_cup(path):
    
    df = pd.read_csv(path, skiprows=7, header = None) 
                     
    df = df.drop([0], axis = 1)
    X = df.iloc[:, :9].values
    
    
    return df, X

In [ ]:
#Opening ML-CUP22-TR and ML-CUP22-TR dataset as useful structures

df, X, Y = open_train_cup("ML-CUP22-TR.csv")
df_blind_test, X_blind_test = open_blind_test_cup("ML-CUP22-TS.csv")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_design, X_test, Y_design, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=0, shuffle=True)

SVM for regression works well with normalized/standardize dataset, so we decide to use StandardScaler that standardize each attribute on X_design to have mean 0 and variance 1. The same scaling must be applied to X_test and X_blind_test to obtain meaningful results.

In [ ]:
from sklearn.preprocessing import StandardScaler  

In [ ]:
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_design)  
X_design = scaler.transform(X_design)  
# apply same transformation to test datasets
X_test = scaler.transform(X_test) 
X_blind_test = scaler.transform(X_blind_test)  

N_design=X_design.shape[0]

now we want to work separately with the 2 target variables

In [ ]:
#splitting of targets

y_1_design = Y_design[:, 0]
y_2_design = Y_design[:, 1]

y_1_test = Y_test[:, 0]
y_2_test = Y_test[:, 1]

In [ ]:
from sklearn.svm import SVR

In [ ]:
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_absolute_error

In [ ]:
def my_grid_search(param_grid, y_design, n):
    grid = GridSearchCV(
        
        SVR(),  # regressor SVM
        param_grid = param_grid, # it changes for each kernel
        cv = RepeatedKFold(n_splits=5, n_repeats=n, random_state=0),
        n_jobs = -1,
        scoring = 'neg_mean_absolute_error',
        refit=True,
        return_train_score=True
        
    )

    return grid.fit(X_design, y_design) # y_design = y_1_design, y_2_design

## Target 1

#### RBF kernel

In [ ]:
# 1) choice of hyperpar.'s ranges to explore

C_interval_rbf = np.logspace(-2, 4, 7)
gamma_interval_rbf = np.logspace(-4, 2, 7) 
eps_interval = np.logspace(-2, 0, 3)


In [ ]:
%%time
# 2) grid search in practice: FIRST RUN THE PREVIOUS CODE!

param_grid_rbf_1= {
    
    'C': C_interval_rbf, 
    'epsilon': eps_interval,
    
    'kernel': ['rbf'],
    
    'gamma': gamma_interval_rbf   
    
}

grid_rbf_1 = my_grid_search(param_grid_rbf_1, y_1_design, 10)

In [ ]:
# definition of useful dataframe

a_rbf_1 = pd.read_csv('cup/a_rbf_1_first.csv')

b_rbf_1 = a_rbf_1[a_rbf_1['rank_test_score'] == 1]

In [ ]:
#useful code for model with 3 hyp. for implementing a heatmap

def heatmap_3(a, b, fix_1, var_1, var_2, score_var):
    
    X = np.unique(b[fix_1].values)  #values for fixed attribute

    for j in range(len(X)):

        plt.figure(j)
        
        matrix = a[(a[fix_1] == X[j])]
        matrix = matrix[[var_1, var_2, score_var]]

        #heatmap in practice
        #glue = matrix.pivot(var_1, var_2, score_var)
        glue = matrix.pivot(var_1, var_2, score_var)
        sns.heatmap(glue, linewidth=.5) # annot=True for values inside the cells
        plt.title('$\epsilon$={}'.format(X[j]))
        plt.tight_layout()
        plt.savefig('{}.pdf'.format(j))
        

        
        
        
    plt.show()  

In [ ]:
# heatmap to visualize graphically the grid search fixing a hyp.(in this case epsilon-> usuful to show its influence on the fsv)

heatmap_3(a_rbf_1, a_rbf_1, 'param_epsilon', 'param_C', 'param_gamma', 'mean_test_score')   #or b_rbf_1

we want to show the expected relationship between C and gamma at given eps (we don't lose generality due to low influence of our values of eps in terms of validation score). For this reason we plot C vs validation score for eps,gamma that provide the best validation score.

In [ ]:
# It looks like gamma and C have a relationship for the highest validation score..let's look at spearman correlation coeff.(r)

from scipy.stats import spearmanr

print('r = {}'.format(spearmanr(a_rbf_1[a_rbf_1['mean_test_score'] >= -0.9][['param_C', 'param_gamma']].values)[0]))

In [ ]:
# useful code for further purposes 

def plot_3(a, fix_1, fix_2, var_1, score_var, scale):

    X = np.unique(a[[fix_1 , fix_2]].values.astype(None), axis=0) # it avoids to repeat the same rows

    for j in range(len(X)): 
        b=a[(a[fix_1] == X[j][0]) & (a[fix_2] == X[j][1])]
        c_best= b[b[score_var] == b[score_var].max()][var_1]
        p_ = (X[j][1]) * c_best
        plt.figure(j)
        plt.plot(
            b[var_1],
            b[score_var], 
        )
        
        plt.scatter( 
            b[var_1],
            b[score_var],
            color='b'
        )
        
        plt.scatter( 
            c_best,
            np.ones(len(c_best))*b[score_var].max(),
            label='best models',
            color='r'
       )
        p_ = (X[j][1]) * c_best
        plt.title('{}={}, {}={}, gamma C={}'.format(fix_1, X[j][0], fix_2, X[j][1], p_))
        plt.xlabel(var_1)
        plt.ylabel(score_var)
        plt.xscale(scale)  #pay attention when you set d
        plt.legend()
        plt.grid()

    plt.show() 

In [ ]:
# 5) (plot) one hyp. vs validatio_accuracy fixing the others hyp. (at values that maximize the validation_accuracy)

score_var='mean_test_score'

fix_1= 'param_epsilon'
fix_2= 'param_gamma'

var_1= 'param_C'

plot_3(a_rbf_1[a_rbf_1['param_epsilon'] == 1.0], fix_1, fix_2, var_1, score_var, 'log')

As the theory suggests lower (than 1) values for gamma require higher values of C to obtain a good trade-off for model flexibility.

In [ ]:
# code for support vectors analysis
# using b_rbf_1, choice of the model with lowest number of support vector
# using a_rb, number of support vector for each model included in the grid search

def sv_analysis_rbf(df, y_design):
    
    hyper_pars = df[['param_C', 'param_epsilon', 'param_gamma']].values
    n_support = []
    model_list = []
    sv_matrix = []

    for j in range(len(hyper_pars)):
        svc = SVR(
            C = hyper_pars[j][0],
            epsilon = hyper_pars[j][1],

            kernel='rbf',

            gamma = hyper_pars[j][2],

        )
        svc.fit(X_design, y_design)

        sv_matrix.append(
            np.array([hyper_pars[j][0], 
                      hyper_pars[j][1], 
                      hyper_pars[j][2],
                      svc.n_support_.sum()/N_design
                     ])
        )

        model_list.append(svc)

    sv_matrix = pd.DataFrame(np.array(sv_matrix))
    #sv_matrix = sv_matrix.rename(mapper={0:'param_C', 1:'param_epsilon', 2:'param_gamma', 3:'fsv'}, axis=1)
    sv_matrix = sv_matrix.rename(mapper={0:'param_C', 1:'param_epsilon', 2:'param_gamma', 3:'fsv'}, axis=1)
    return sv_matrix, model_list

In [ ]:
%%time
# number of support vector for each model included in the grid search fixed a hyp. **by heatmap**

sv_a, sv_b = sv_analysis_rbf(a_rbf_1, y_1_design)[0], sv_analysis_rbf(b_rbf_1, y_1_design)[0]

score_var='fsv'

fix_1 = 'param_epsilon'

var_1= 'param_C'
var_2= 'param_gamma'

heatmap_3(sv_a, sv_a, fix_1, var_1, var_2, score_var)

In [ ]:
#  print the performance of the best model 

N_design = X_design.shape[0] 

rbf_final_1 = grid_rbf_1.best_estimator_

print('best model choosen:\n{}'.format(rbf_final_1))

#print('number of support vectors:\n{}'.format(rbf_final_1.n_support_.sum()))
print('fraction of support vectors:\n{}'.format(round_it(rbf_final_1.n_support_.sum()/N_design,2)))

print('validation MAE:\n{}'.format(-round_it(b_rbf_1.iloc[np.argmin(sv_b['fsv'])]['mean_test_score'], 3)) )

y_pred_1 = rbf_final_1.predict(X_test)
print('test MAE:\n{}'.format(round_it(mean_absolute_error(y_1_test, y_pred_1), 4)) )

### Sigmoidal kernel

In [ ]:
# 1) choice of hyperpar.'s ranges to explore

C_interval_sigmoid = np.logspace(-3, 3, 7)
gamma_interval_sigmoid = np.logspace(-3, 3, 7)
coef0_interval_sigmoid = np.logspace(-4, -2, 3)
eps_interval = np.logspace(-1, 0, 2)


In [ ]:
%%time
# 2) grid search in practice: FIRST RUN THE PREVIOUS CODE!

param_grid_sigmoid_1= {
    
    'C': C_interval_sigmoid,  
    'epsilon': eps_interval,
    
    'kernel': ['sigmoid'],  
    
    'gamma': gamma_interval_sigmoid, 
    'coef0': coef0_interval_sigmoid, 
    
}

grid_sigmoid_1 = my_grid_search(param_grid_sigmoid_1, y_1_design, 10)

In [ ]:
# definition of useful dataframe

#a_sigmoid_1 = pd.DataFrame(grid_sigmoid_1.cv_results_)
a_sigmoid_1 = pd.read_csv('cup/a_sigmoid_1_first.csv')

b_sigmoid_1 = a_sigmoid_1[a_sigmoid_1['rank_test_score'] == 1]

In [ ]:
def heatmap_4(a, b, fix_1, fix_2, var_1, var_2, score_var):



    X = np.unique(b[[fix_1, fix_2]].values.astype(None), axis=0)

    for j in range(len(X)):
        plt.figure(j)

        matrix = a[(a[fix_1] == X[j][0]) & (a[fix_2] == X[j][1])]
        matrix = matrix[[var_1, var_2, score_var]]
        
        glue = matrix.pivot(var_1, var_2, score_var)
        sns.heatmap(glue, cmap="crest", linewidth=.5)
        plt.title('{}={}, {}={}'.format(fix_1, X[j][0], fix_2, X[j][1]))

    plt.show()  


In [ ]:
heatmap_4(a_sigmoid_1, a_sigmoid_1 ,'param_coef0', 'param_epsilon', 'param_C', 'param_gamma', 'mean_test_score')

In [ ]:
def sv_analysis_sigmoid(df, y):
    
    hyper_pars = df[['param_C', 'param_epsilon', 'param_gamma', 'param_coef0']].values
    model_list = []
    sv_matrix = []

    for j in range(len(hyper_pars)):
        svr = SVR(
            C = hyper_pars[j][0],
            epsilon = hyper_pars[j][1],

            kernel='sigmoid',

            gamma = hyper_pars[j][2],
            coef0 = hyper_pars[j][3],

        )
        svr.fit(X, y)

        sv_matrix.append(
            np.array([hyper_pars[j][0], 
                      hyper_pars[j][1], 
                      hyper_pars[j][2],
                      hyper_pars[j][3],
                      round(svr.n_support_.sum()/N_design, 2)
                     ])
        )

        model_list.append(svr)

    sv_matrix = pd.DataFrame(np.array(sv_matrix))
    sv_matrix = sv_matrix.rename(mapper={
        0:'param_C', 1:'param_epsilon', 2:'param_gamma', 3:'param_coef0', 4:'fsv'}, axis=1)
    return sv_matrix, model_list

In [ ]:
# 7.a) number of support vector for each model included in the grid search **by heatmap**

sv_a = sv_analysis_sigmoid(a_sigmoid_1, X_design, y_1_design)[0]
sv_b = sv_analysis_sigmoid(b_sigmoid_1, X_design, y_1_design)[0]

score_var='fsv'

fix_1 = 'param_epsilon'
fix_2 = 'param_coef0'

var_1= 'param_C'
var_2= 'param_gamma'

heatmap_4(sv_a, sv_a, fix_1, fix_2, var_1, var_2, score_var)

In [ ]:
# 8) choice of the best model 

N_design = X_design.shape[0] 

sv_b, model_list_b = sv_analysis_sigmoid(b_sigmoid_1, X_design, y_1_design)

sigmoid_final = grid_sigmoid_1.best_estimator_

print('best model choosen:\n{}'.format(sigmoid_final))

#print('number of support vectors:\n{}'.format(sigmoid_final.n_support_.sum()))
print('fraction of support vectors:\n{}'.format(round_it(sigmoid_final.n_support_.sum()/N_design,2)))

print('validation MAE:\n{}'.format(-round_it(b_sigmoid_1.iloc[np.argmin(sv_b['fsv'])]['mean_test_score'], 3)) )

y_pred_1 = sigmoid_final.predict(X_test)
print('test MAE:\n{}'.format(round_it(mean_absolute_error(y_1_test, y_pred_1), 4)))

## Insights 

We want to show the relationship between the size of the training set and the fsv(fraction of support vectors)

In [ ]:
X_sv = train_test_split(
    X_design, y_1_design, test_size=0.2, random_state=0, shuffle=True)[1]

y_sv = train_test_split(
    X_design, y_1_design, test_size=0.2, random_state=0, shuffle=True)[3]

In [ ]:

sv_a, sv_b = sv_analysis_sigmoid(a_sigmoid_1, X_sv, y_sv)[0], sv_analysis_sigmoid(b_sigmoid_1, X_sv, y_sv)[0]

score_var='fsv'

fix_1 = 'param_epsilon'
fix_2 = 'param_coef0'

var_1= 'param_C'
var_2= 'param_gamma'

heatmap_4(sv_a, sv_a, fix_1, fix_2, var_1, var_2, score_var)

Does the previous approach make sense?

In [ ]:
def fsv_train_sigmoid(X , y, C, eps, gamma, coef0):
    
    fsv_list=[]
    axis_list=[]
    
    for n in range(1, 10):
        
        #creation of variable training set
        x_sv = train_test_split(
        X, y, test_size=n/10, random_state=0, shuffle=True)[1]

        y_sv = train_test_split(
        X, y, test_size=n/10, random_state=0, shuffle=True)[3]
        
        N_sv=x_sv.shape[0]
        #set hyp.
        svr = SVR(
            C = C ,
            epsilon = eps,

            kernel= 'sigmoid',

            gamma = gamma,
            coef0 = coef0,

        )
        #fit the model with variable set
        svr.fit(x_sv, y_sv)
        
        axis_list.append(n/10)
        fsv_list.append(svr.n_support_.sum()/N_sv)
        
        
    fsv_list=np.array(fsv_list)
    plt.plot(axis_list, fsv_list)
    
    m=fsv_list.mean()
    sigma=fsv_list.std()
    
    plt.plot([k/10 for k in range(1,10)], np.ones(len(fsv_list))*m, color='r', 
             linestyle='--', label='fsv = {}+-{}'.format(round_it(m,2), round_it(sigma,1)))

#     plt.plot([k/10 for k in range(1,10)], np.ones(len(fsv_list))*(m+sigma), color='r', linestyle='--')
#     plt.plot([k/10 for k in range(1,10)], np.ones(len(fsv_list))*(m-sigma), color='r', linestyle='--')
    
    plt.ylabel('fsv')
    plt.xlabel('fraction of the entire design set')
    plt.legend()
    plt.grid()
    
    print('fsv = {}+-{}'.format(round_it(m,2), round_it(sigma,1)))

    plt.show()
        
    

In [ ]:
#
C=1000
eps=1
gamma=3e-6
coef0=0.2

fsv_train_sigmoid(X_design , y_1_design, C, eps, gamma, coef0)

We want to provide a final model trained on the entire X_design. However, can we check the sv for each model of the GS on a fraction of X_design? 

Let us consider the previous. For those input values we obtain interesting observation: starting  from (0.1xlen(X_design), **fsv=0.9** ) to 
(0.9xlen(X_design), **fsv=0.5**). So to have a complete view of support vectors for each model(cell of GridSearch) is more reliable train the models on the entire design test. Or anyway train the final model on the same fraction of X_design used for the check phase.

## Target 2

### RBF kernel

In [ ]:
# 1) choice of hyperpar.'s ranges to explore

C_interval_rbf = np.logspace(-2, 4, 7)
gamma_interval_rbf = np.logspace(-4, 2, 7) 
epsilon_interval = np.logspace(-2, 0, 3)


In [ ]:
%%time
# 2) grid search in practice: FIRST RUN THE PREVIOUS CODE!
param_grid_rbf_2 = {
    'C': C_interval_rbf, #must be strictly positive  
    'epsilon': eps_interval,
    
    'kernel': ['rbf'],
    
    'gamma': gamma_interval_rbf #gamma must be greater than 0      
    
}

grid_rbf_2 = my_grid_search(param_grid_rbf_2, y_2_design, 10)

In [ ]:
# definition of useful dataframe

a_rbf_2 = pd.DataFrame(grid_rbf_2.cv_results_)

b_rbf_2 = a_rbf_2[a_rbf_2['rank_test_score'] == 1]  #cv_results_ restricted to model with best MAE 

In [ ]:
# heatmap to visualize graphically the grid search fixing a hyp.

heatmap_3(a_rbf_2, a_rbf_2, 'param_epsilon', 'param_C', 'param_gamma', 'mean_test_score')

- SVR for target 2 works worse than target 1 with the same coarse GridSearch

- same reasoning done for target 1: pretty similiar GrisSearch for different values of eps, maybe for eps=1 is a bit worse. BUT taking into account the nsv it's more reasonable in terms of efficiency focus our attention on eps=1. 

In [ ]:
from scipy.stats import spearmanr

print('r = {}'.format(spearmanr(a_rbf_1[a_rbf_1['mean_test_score'] >= -0.9][['param_C', 'param_gamma']].values)[0]))

In [ ]:
# 5) (plot) one hyp. vs validatio_accuracy fixing the others hyp. (at values that maximize the validation_accuracy)

score_var='mean_test_score'

fix_1= 'param_epsilon'
fix_2= 'param_gamma'

var_1= 'param_C'

plot_3(a_rbf_1[a_rbf_1['param_epsilon'] == 1.0], fix_1, fix_2, var_1, score_var, 'log')

In [ ]:
%%time
# 7.a) number of support vector for each model included in the grid search **by heatmap**

sv_a, sv_b = sv_analysis_rbf(a_rbf_2, y_2_design)[0], sv_analysis_rbf(b_rbf_2, y_2_design)[0]

score_var='n_support_vectors'

fix_1 = 'param_epsilon'

var_1= 'param_C'
var_2= 'param_gamma'

heatmap_3(sv_a, sv_a, fix_1, var_1, var_2, score_var)

In [ ]:
# 8) choice of the model with lowest number of support vector

N_design = X_design.shape[0] 

sv_b, model_list_b = sv_analysis_rbf(b_rbf_2, y_2_design)

rbf_final_2 = grid_rbf_2.best_estimator_

print('best model choosen:\n{}'.format(rbf_final_2))
print('number of support vectors:\n{}'.format(rbf_final_2.n_support_.sum()))
print('fraction of support vectors:\n{}'.format(round_it(rbf_final_2.n_support_.sum()/N_design,2)))

print('validation MAE:\n{}'.format(-round_it(b_rbf_2.iloc[np.argmin(sv_b['fsv'])]['mean_test_score'], 4)) )

y_pred_2 = rbf_final_2.predict(X_test)
print('test MAE:\n{}'.format(round_it(mean_absolute_error(y_2_test, y_pred_2), 4)) )

### Sigmoid kernel

In [ ]:
# 1) choice of hyperpar.'s ranges to explore

C_interval_sigmoid = np.logspace(3, 4, 7)
gamma_interval_sigmoid = np.logspace(-4, -3, 7)
coef0_interval_sigmoid = np.logspace(-3, -1, 3)
eps_interval = [1.0]


In [ ]:
%%time
# 2)

param_grid_sigmoid_2={
    
    'C': C_interval_sigmoid,  
    'epsilon': eps_interval,
    
    'kernel': ['sigmoid'],  
    
    'gamma': gamma_interval_sigmoid, 
    'coef0': coef0_interval_sigmoid,  
}

grid_sigmoid_2 = my_grid_search(param_grid_sigmoid_2, y_2_design, 10)


In [ ]:
%%time
# 2)

param_grid_sigmoid_2={
    
    'C': [1e3],  
    'epsilon': [1],
    
    'kernel': ['sigmoid'],  
    
    'gamma': [1e-4], 
    'coef0': [1e-3],  
}

grid_sigmoid_2 = my_grid_search(param_grid_sigmoid_2, y_2_design, 10)


In [ ]:
# definition of useful dataframe

a_sigmoid_2 = pd.DataFrame(grid_sigmoid_2.cv_results_)
#a_sigmoid_2 = pd.read_csv('cup/a_sigmoid_2_first.csv')

b_sigmoid_2 = a_sigmoid_2[a_sigmoid_2['rank_test_score'] == 1]


In [ ]:
heatmap_4(
    a_sigmoid_2, 
    a_sigmoid_2 ,
    'param_coef0', 
    'param_epsilon', 
    'param_C', 
    'param_gamma', 
    'mean_test_score'
)

In [ ]:
# 7.a) number of support vector for each model included in the grid search **by heatmap**

sv_a = sv_analysis_sigmoid(a_sigmoid_2, X_design, y_2_design)[0]
sv_b = sv_analysis_sigmoid(b_sigmoid_2, X_design, y_2_design)[0]

score_var='fsv'

fix_1 = 'param_epsilon'
fix_2 = 'param_coef0'

var_1= 'param_C'
var_2= 'param_gamma'

heatmap_4(sv_a, sv_a, fix_1, fix_2, var_1, var_2, score_var)

for the last GS we notice very well the relationship between areas with higher validation score and lower nsv.

In [ ]:
# 8) choice of the best model 

N_design = X_design.shape[0] 

sv_b, model_list_b = sv_analysis_sigmoid(b_sigmoid_2, X_design, y_2_design)

sigmoid_final = grid_sigmoid_2.best_estimator_

print('best model choosen:\n{}'.format(sigmoid_final))

print('number of support vectors:\n{}'.format(sigmoid_final.n_support_.sum()))
print('fraction of support vectors:\n{}'.format(round_it(sigmoid_final.n_support_.sum()/N_design, 2)))

print('validation MAE:\n{}'.format(-round_it(b_sigmoid_2.iloc[np.argmin(sv_b['fsv'])]['mean_test_score'], 4)) )

y_pred = sigmoid_final.predict(X_test)
print('test MAE:\n{}'.format(round_it(mean_absolute_error(y_2_test, y_pred), 4)))

## Conclusion

Clearly from the previous output we observe that the best results come out from RBF kernel. Taking into account both the target variable let's compute the final MEE for the SVR.

In [ ]:
e_1=np.square(y_1_test - rbf_final_1.predict(X_test))
e_2=np.square(y_2_test - rbf_final_2.predict(X_test))
              
MEE_final = (1/X_test.shape[0])*np.sqrt(e_1 + e_2).sum()
print('MEE={}'.format(round(MEE_final, 2)))